In [1]:
import numpy as np

output = {}
for i in range(1000):
    n = np.random.choice(6)
    if not n in output:
        output[n] = 0
    output[n] += 1
output

In [2]:
def roll_dice():
    return np.random.choice(6) + 1

In [3]:
def roll_all_dice(count):
    return sorted([roll_dice() for i in range(count)], reverse=True)
roll_all_dice(2)

[6, 2]

In [4]:
def attack_once(attacker_dice_count, defender_dice_count):
    attacker_dice = roll_all_dice(attacker_dice_count)
    defender_dice = roll_all_dice(defender_dice_count)
    
    attacker_loss = 0
    defender_loss = 0
    for i in range(min(len(attacker_dice), len(defender_dice))):
        attacker_roll = attacker_dice[i]
        defender_roll = defender_dice[i]
        if attacker_roll > defender_roll:
            defender_loss += 1
        else:
            attacker_loss += 1
    
    return (attacker_loss, defender_loss), (attacker_dice, defender_dice)
    

# Returns (loss, dice)
attack_once(3, 2)
    

((0, 2), ([5, 5, 3], [2, 2]))

In [21]:
def get_max_dice_count(soldier_count, is_attacker = False):
    max_dice = 3 if is_attacker else 2
    
    return min(max_dice, max(soldier_count - 1, 1))
get_max_dice_count(3, False)

2

In [6]:
def attack_with_all_dice(attacker_soldier_count, defender_soldier_count):
    attacker_dice_count = get_max_dice_count(attacker_soldier_count, True)
    defender_dice_count = get_max_dice_count(defender_soldier_count, False)
    if attacker_dice_count == 0:
        return None
    if defender_dice_count == 0:
        return None
    
    return attack_once(attacker_dice_count, defender_dice_count)
#     print(attacker_dice_count, defender_dice_count)
attack_with_all_dice(3, 1)

((0, 1), ([5, 3], [1]))

In [22]:
# Continue to attack as long as possible
def blitz(attacker_soldier_count, defender_soldier_count):
    while(attacker_soldier_count > 1 and defender_soldier_count > 0):
        loss, dice = attack_with_all_dice(attacker_soldier_count, defender_soldier_count)

        attacker_loss, defender_loss = loss
        attacker_soldier_count -= attacker_loss
        defender_soldier_count -= defender_loss        
    if attacker_soldier_count > 1:
        return [1, 0]
    return [0, 1]
blitz(10, 10)

[1, 0]

In [8]:
def perform_analysis(attacker_soldier_count, defender_soldier_count, iterations=100):
    attacker_win_count = 0
    total_attack_count = 0
    
    for i in range(iterations):
        attacker_win, defender_win = blitz(attacker_soldier_count, defender_soldier_count)
        attacker_win_count += attacker_win
        total_attack_count += 1
    
    percent = 0
    if total_attack_count == 0:
        percent == 100
    else:
        ratio = (attacker_win_count / total_attack_count)
        percent = np.round(ratio * 100, 2)
    print('Attacker won {}% of the time'.format(percent))

perform_analysis(3, 2)

Attacker won 43.0% of the time


In [9]:
def perform_all_analysis():
    ITERATIONS = 10000
    soldier_counts = [
        (2, 1),
        (3, 1),
        (4, 1),
        (5, 1),
        (3, 2),
        (2, 2),
        (4, 3),
        (6, 2),
        (10, 10),
    ]
    
    for soldier_count in soldier_counts:
        attacker_soldiers, defender_soldiers = soldier_count
        print('For ({}, {})'.format(attacker_soldiers, defender_soldiers), end=': ')
        perform_analysis(attacker_soldiers, defender_soldiers, iterations=ITERATIONS)
perform_all_analysis()

For (2, 1): Attacker won 41.64% of the time
For (3, 1): Attacker won 75.54% of the time
For (4, 1): Attacker won 91.97% of the time
For (5, 1): Attacker won 96.87% of the time
For (3, 2): Attacker won 51.82% of the time
For (2, 2): Attacker won 17.18% of the time
For (4, 3): Attacker won 51.11% of the time
For (6, 2): Attacker won 96.14% of the time
For (10, 10): Attacker won 50.92% of the time


In [23]:
ITERATIONS=10000
attacker_soldiers=9
defender_soldiers=6
perform_analysis(attacker_soldiers, defender_soldiers, iterations=ITERATIONS)

Attacker won 76.5% of the time


In [50]:
perform_analysis(3, 1, 10000000)

Attacker won 75.41% of the time


# Chance of Attacker Defeating Defender

Attacking dice has a 15/36 (roughly 41.67%) chance of defeating a defending dice.

In [11]:
chance_of_winning = 15 / 36
chance_of_losing = 1 - ( 15 / 36 )


In [12]:
# Chance of winning when attacker has 3 dice and defender has two dice
def chance_of_attacker_winning(dice_count):
    return 1 - (chance_of_losing ** dice_count)

chance_of_attacker_winning(3)

# But... this does not reflect an actual battle, because the attacker can attack multiple times even if they lose the first time


0.8015046296296298

In [13]:
# Chance of winning when allowed to roll multiple times
def chance_of_attacker_winning_depleting_all_dice(dice_count):
    chance_of_losing_all = 1
    for this_dice_count in range(dice_count, 0, -1): # Up to 3
        chance_of_winning_this_round = chance_of_attacker_winning(this_dice_count)
        print('chance_of_winning_this_round', chance_of_winning_this_round, chance_of_losing_all)
        chance_of_losing_this_round = 1 - chance_of_winning_this_round
        chance_of_losing_all *= chance_of_losing_this_round
    
    chance_of_winning_any = 1 - chance_of_losing_all
    
    return chance_of_winning_any

chance_of_attacker_winning_depleting_all_dice(2)


chance_of_winning_this_round 0.6597222222222223 1
chance_of_winning_this_round 0.41666666666666674 0.3402777777777777


0.8015046296296298

In [14]:
perform_analysis(4, 1, 100000)

Attacker won 91.63% of the time


In [15]:
perform_analysis(3, 1, 100000)

Attacker won 75.6% of the time


## Using Statistical Probability

In [ ]:
def calculate_winning_dividend(numbers_count=4):
    r

In [ ]:
NUMBERS_ON_DICE=3
ATTACKER_DICE_COUNT=1
DEFENDER_DICE_COUNT=1

chance_of_winning_attack = 

In [29]:
1 - ((1/2) ** 3) - ((3/6) * ((1/2) ** 3))

0.8125

In [30]:
0.8125 * 64

52.0

In [32]:
((3/6) * ((1/2) ** 3)) * 64

4.0

In [34]:
((1/2) ** 3) * 8

1.0

In [35]:
ATTACKER_DICE_COUNT=3

attacker_dice_count = ATTACKER_DICE_COUNT
probability_defender_wins = (1/2) ** 3
probability_of_tie = (1/2) ** 3

probability_attacker_wins = 1 - probability_defender_wins - probability_of_tie
probability_attacker_wins

0.75